# 📜 Abstract

This project builds a **Financial Content Assistant** using Retrieval-Augmented Generation (RAG) architecture.  
The system allows users to upload financial documents (PDFs, TXT files, spreadsheets) and ask natural language questions.  
It intelligently retrieves relevant document sections and generates coherent, contextual answers using OpenAI’s models.  

Key Features:
- **Multi-file support:** PDF, TXT, CSV, XLSX.
- **Metadata extraction:** Date, Company Name, Document Type (e.g., 10-K, 10-Q).
- **Document chunking:** Optimized retrieval granularity.
- **Semantic Search:** Vector database-backed document retrieval.
- **Interactive Assistant Interface:** User-friendly UI with badges for question type detection (e.g., Investment, Financial Analysis, etc.).
- **Evaluation Metrics:** Assess answer quality and source attribution.

This project demonstrates the application of advanced natural language processing techniques for intelligent financial document querying and content understanding.


# 🧾 Financial Content Assistant - Project Notebook

Welcome! This notebook walks through the complete process of building a Financial Content Assistant using Retrieval-Augmented Generation (RAG) with LangChain and OpenAI models.

## Table of Contents
1. 📦 [Setup and Installations](#Setup-and-Installations)
2. 🔑 [Configuration](#Configuration)
3. 📄 [Uploading and Processing Documents](#Uploading-and-Processing-Documents)
4. 🧠 [Metadata Extraction](#Metadata-Extraction)
5. ✂️ [Splitting Documents](#Splitting-Documents)
6. 🗃️ [Creating Vector Database](#Creating-Vector-Database)
7. 🔗 [Retrieval-Augmented Generation (RAG) Chain](#Retrieval-Augmented-Generation-RAG-Chain)
8. 📊 [Evaluation Metrics](#Evaluation-Metrics)
9. 🧪 [Basic Testing](#Basic-Testing)
10. 🚀 [Full Pipeline Execution](#Full-Pipeline-Execution)
11. 💬 [Interactive Assistant Interface](#Interactive-Assistant-Interface)
12. 🎯 [Final Launch](#Final-Launch)
13. 💾 [Save Progress](#Save-Progress)

---



# 📦 Setup and Installations
<a id="Setup-and-Installations"></a>

This section installs and imports necessary libraries for Financial Content Assistant.


In [2]:
# Install necessary libraries
!pip install langchain langchain-openai langchain-community openai faiss-cpu tiktoken pypdf
!pip install sentence-transformers
!pip install ipywidgets
!pip install pypdf
!pip install -U langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211

In [3]:
# Import Required Libraries
import os
import getpass
import tempfile
from google.colab import files
from IPython.display import clear_output
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 🔑 Configuration
This section sets up API keys and environment variables.


In [4]:
# Set OpenAI API Key
import os
import getpass

# Securely input your API key (it won't be visible in the notebook)
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

# Verify the key is set (this will only show "True" or "False", not the actual key)
print(f"API key is set: {bool(os.environ.get('OPENAI_API_KEY'))}")

Enter your OpenAI API Key: ··········
API key is set: True



# 📄 Uploading and Processing Documents
Upload your financial documents (PDF, TXT, CSV, XLSX) and prepare them for analysis.
- Automatically detects file types
- Handles different encodings
- Converts spreadsheets to plain text


In [74]:
def upload_and_process_documents():
    """Upload and process financial documents (TXT, PDF, CSV, XLSX)"""
    print("Upload your financial documents (TXT, PDF, CSV, or XLSX files)")
    uploaded = files.upload()

    documents = []

    for filename, content in uploaded.items():
        try:
            # Create a temporary file
            _, file_extension = os.path.splitext(filename)
            file_extension = file_extension.lower()
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=file_extension)

            with open(temp_file.name, 'wb') as f:
                f.write(content)

            # Process based on file type
            if file_extension == '.pdf':
                print(f"Processing PDF: {filename}")
                loader = PyPDFLoader(temp_file.name)
                file_docs = loader.load()

                for doc in file_docs:
                    doc.metadata["source"] = filename
                    # Inside your document processing function, add:
                    metadata = extract_metadata(doc, filename)
                    doc.metadata.update(metadata)  # Add the extracted metadata

                documents.extend(file_docs)
                print(f"Successfully loaded PDF: {filename} ({len(file_docs)} pages)")

            elif file_extension == '.txt':
                print(f"Processing TXT: {filename}")
                successful = False
                for encoding in ['utf-8', 'latin-1', 'cp1252']:
                    try:
                        with open(temp_file.name, 'r', encoding=encoding) as f:
                            text = f.read()

                        doc = Document(
                            page_content=text,
                            metadata={"source": filename}
                        )
                        metadata = extract_metadata(doc, filename)
                        doc.metadata.update(metadata)

                        documents.append(doc)
                        print(f"Successfully loaded TXT: {filename} with {encoding} encoding")
                        successful = True
                        break
                    except UnicodeDecodeError:
                        continue
                if not successful:
                    print(f"Failed to decode {filename} with all attempted encodings")

            elif file_extension in ['.csv', '.xlsx', '.xls']:
                print(f"Processing spreadsheet: {filename}")

                if file_extension == '.csv':
                    df = pd.read_csv(temp_file.name)
                else:
                    df = pd.read_excel(temp_file.name)

                text = df.to_string(index=False)
                doc = Document(
                    page_content=text,
                    metadata={"source": filename}
                )
                metadata = extract_metadata(doc, filename)
                doc.metadata.update(metadata)

                documents.append(doc)
                print(f"Successfully loaded spreadsheet: {filename}")

            else:
                print(f"Unsupported file type: {file_extension} for {filename}")

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

        finally:
            # Always clean up the temporary file
            try:
                os.unlink(temp_file.name)
            except Exception as cleanup_error:
                print(f"Error deleting temp file: {cleanup_error}")

    return documents



# 🧠 Metadata Extraction
Extract metadata like date, company name, and document type from uploaded content.


In [75]:
def extract_metadata(document, filename):
    """Extract and add richer metadata to documents"""
    metadata = {"source": filename}

    # Extract date information if available
    import re
    date_patterns = [
        r'(\d{4}-\d{2}-\d{2})',  # YYYY-MM-DD
        r'(\d{2}/\d{2}/\d{4})',  # MM/DD/YYYY
        r'(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}'  # Month DD, YYYY
    ]

    for pattern in date_patterns:
        matches = re.findall(pattern, document.page_content[:1000])  # Look in first 1000 chars
        if matches:
            metadata["date"] = matches[0]
            break

    # Extract company name if available
    company_patterns = [
        r'([A-Z][A-Za-z]+,?\s+Inc\.)',
        r'([A-Z][A-Za-z]+\s+Corporation)',
        r'([A-Z][A-Za-z]+\s+Company)'
    ]

    for pattern in company_patterns:
        matches = re.findall(pattern, document.page_content[:1000])
        if matches:
            metadata["company"] = matches[0]
            break

    # Add document type
    if "10-K" in filename or "10K" in filename:
        metadata["doc_type"] = "Annual Report"
    elif "10-Q" in filename or "10Q" in filename:
        metadata["doc_type"] = "Quarterly Report"

    return metadata

# ✂️ Splitting Documents
Split large documents into smaller manageable chunks:
- Chunk Size: 1000 characters
- Chunk Overlap: 200 characters

Essential for high quality semantic search and RAG performance.

# 🗃️ Creating Vector Database
Use document embeddings to create a searchable vector store for fast semantic retrieval.


In [86]:
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into chunks for processing"""
    if not documents:
        print("No documents were successfully loaded.")
        return []

    # Split documents
    print("\nSplitting documents into chunks...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Created {len(chunks)} chunks from {len(documents)} documents/pages")
    return chunks

def create_vector_db(chunks):
    """Create a vector database from document chunks"""
    if not chunks:
        print("No chunks to create vector database with.")
        return None

    # Create vector store with in-memory persistence
    print("\nCreating vector database...")
    embeddings = OpenAIEmbeddings()

    # Use FAISS for in-memory storage to avoid SQLite issues in Colab
    # vector_db = Chroma.from_documents(
    #     documents=chunks,
    #     embedding=embeddings,
    #     persist_directory=None  # This forces in-memory storage
    # )
    vector_db = FAISS.from_documents(chunks, embeddings) # Changed to FAISS

    print("Vector database created successfully!")
    return vector_db

# 🔗 Retrieval-Augmented Generation (RAG) Chain
Set up a LangChain chain that:
- Retrieves top relevant chunks from the vector database
- Passes them to OpenAI model to generate a final answer


In [82]:
def create_rag_chain(vector_db):
    """Create a RAG-based question answering chain"""
    if not vector_db:
        print("No vector database provided to create RAG chain!")
        return None

    # Create QA chain
    print("\nCreating question-answering chain...")
    template = """
    You are a sophisticated financial analyst specializing in explaining financial information
    from company reports and financial documents.

    Use the following pieces of context to answer the user's question about financial data.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    When describing financial information, be precise about numbers and date periods.
    Clearly indicate which financial metrics come from which reporting periods.

    Context:
    {context}

    Question: {question}

    Answer:
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_db.as_retriever(search_kwargs={"k": 4}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )

    print("RAG chain created successfully!")
    return qa_chain

# 📊 Evaluation Metrics
Functions to assess:
- Answer correctness
- Source reliability
- Relevance
Designed for improving system performance systematically.


In [83]:
def evaluate_response_quality(question, answer, sources):
    """Evaluate the quality of RAG responses"""
    # Use ChatGPT to evaluate response quality
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    prompt = f"""
    Evaluate the quality of this financial response:

    Question: {question}

    Answer: {answer}

    Sources: {sources}

    Rate on a scale of 1-10:
    1. Accuracy (how factually correct)
    2. Completeness (how comprehensive)
    3. Relevance (how well it addresses the question)
    4. Clarity (how easy to understand)

    Format your response as JSON:
    {{
        "accuracy": X,
        "completeness": X,
        "relevance": X,
        "clarity": X,
        "average": X,
        "feedback": "brief feedback"
    }}
    """

    try:
        response = llm.invoke(prompt)
        evaluation = json.loads(response.content)
        return evaluation
    except:
        return {
            "accuracy": 0,
            "completeness": 0,
            "relevance": 0,
            "clarity": 0,
            "average": 0,
            "feedback": "Error evaluating response"
        }

# 🧪 Basic Testing
Run quick tests on the Assistant to validate pipeline correctness.


In [84]:
def test_financial_assistant(qa_chain, question):
    """Test the financial assistant with a sample question"""
    if not qa_chain:
        print("No QA chain available!")
        return

    print(f"\nQuestion: {question}")
    print("\nSearching and generating response...")

    try:
        result = qa_chain({"query": question})

        print("\n======= ANSWER =======")
        print(result["result"])
        print("\n======= SOURCES =======")

        sources_seen = set()
        for i, doc in enumerate(result["source_documents"][:3]):  # Show top 3 sources
            source = doc.metadata.get("source", "Unknown")
            page = doc.metadata.get("page", "")
            page_info = f" (Page {page})" if page != "" else ""

            if source not in sources_seen:
                sources_seen.add(source)
                print(f"- {source}{page_info}")

    except Exception as e:
        print(f"Error generating response: {str(e)}")

# 🚀 Full Pipeline Execution
Upload → Metadata → Chunk → Embed → RAG → Query

The entire system comes together here!


In [87]:
# Main execution flow
print("==== Financial Content Assistant Setup ====")
print("This assistant helps answer financial questions based on your documents\n")

# Step 1: Upload and process documents
print("Step 1: Upload and process documents")
documents = upload_and_process_documents()

if not documents:
    print("No documents were loaded. Please run this cell again and upload documents.")
else:
    # Step 2: Split documents into chunks
    print("\nStep 2: Creating document chunks for semantic search")
    chunks = split_documents(documents)

    if not chunks:
        print("No document chunks were created. Please check your documents.")
    else:
        # Step 3: Create vector database
        print("\nStep 3: Creating vector database for semantic search")
        vector_db = create_vector_db(chunks)

        # Step 4: Create RAG chain
        print("\nStep 4: Creating the RAG-based question answering system")
        qa_chain = create_rag_chain(vector_db)

        print("\n==== Setup Complete! ====")
        print("You can now test your financial assistant with a sample question.")

        # Optional: Test with a sample question
        sample_question = "What are the main financial highlights from these documents?"
        print("\nTesting with a sample question:")
        test_financial_assistant(qa_chain, sample_question)

        print("\nSetup complete! Run the interactive interface below to ask more questions.")

==== Financial Content Assistant Setup ====
This assistant helps answer financial questions based on your documents

Step 1: Upload and process documents
Upload your financial documents (TXT, PDF, CSV, or XLSX files)


Saving Netflix 10-k.pdf to Netflix 10-k (2).pdf
Processing PDF: Netflix 10-k (2).pdf
Successfully loaded PDF: Netflix 10-k (2).pdf (72 pages)

Step 2: Creating document chunks for semantic search

Splitting documents into chunks...
Created 389 chunks from 72 documents/pages

Step 3: Creating vector database for semantic search

Creating vector database...
Vector database created successfully!

Step 4: Creating the RAG-based question answering system

Creating question-answering chain...
RAG chain created successfully!

==== Setup Complete! ====
You can now test your financial assistant with a sample question.

Testing with a sample question:

Question: What are the main financial highlights from these documents?

Searching and generating response...

======= ANSWER =======
The main financial highlights from the documents provided are as follows:

1. Streaming revenues for the year ended December 31, 2024, were $39,000,966, showing a 16% increase from $33,640,458 in 2023.
2. DVD revenue

# 💬 Interactive Assistant Interface
Run the enhanced chatbot UI:
- Detects financial question type
- Specialized visual formatting (badges)
- Chat history summary and visualization



In [88]:
# Interactive Interface
def interactive_financial_assistant(qa_chain):
    """Interactive interface for the financial assistant"""
    if not qa_chain:
        print("No QA chain available. Please set up the assistant first.")
        return

    print("\n===== Financial Content Assistant =====")
    print("Type your financial questions below. Type 'exit' to quit.")

    while True:
        question = input("\nYour question: ")

        if question.lower() in ['exit', 'quit', 'q']:
            print("Thank you for using the Financial Content Assistant!")
            break

        if not question.strip():
            continue

        try:
            # Clear previous output for better readability
            clear_output(wait=True)

            print("===== Financial Content Assistant =====")
            print(f"Question: {question}")
            print("\nProcessing...")

            result = qa_chain({"query": question})

            print("\nAnswer:")
            print(result["result"])

            print("\nSources:")
            sources_seen = set()
            for doc in result["source_documents"][:3]:  # Show top 3 sources
                source = doc.metadata.get("source", "Unknown")
                page = doc.metadata.get("page", "")
                page_info = f" (Page {page})" if page != "" else ""

                if source not in sources_seen:
                    sources_seen.add(source)
                    print(f"- {source}{page_info}")

            print("\nType your next question or 'exit' to quit.")

        except Exception as e:
            print(f"Error: {str(e)}")
            print("\nType your next question or 'exit' to quit.")


# 🎯 Final Launch
Launch the enhanced financial assistant with all features activated.


In [89]:
# Run the interactive assistant
try:
    # This will only run if qa_chain exists from the previous cell
    interactive_financial_assistant(qa_chain)
except NameError:
    print("You need to run the setup cell first to create the qa_chain.")
    print("Please run the previous cell and then run this cell again.")

===== Financial Content Assistant =====
Question: what is the net profit 

Processing...

Answer:
The net profit for Netflix, Inc. for the year ended December 31, 2024, was $8,711,631,000.

Sources:
- Netflix 10-k (2).pdf (Page 38)

Type your next question or 'exit' to quit.

Your question: quit
Thank you for using the Financial Content Assistant!


##Now that we've  successfully built the core RAG functionality.

##Let's enhance it with a polished UI, additional features, and testing capabilities.

##Here's a step-by-step plan that continues

In [90]:
# 📦 Import Required Libraries
from IPython.display import clear_output, HTML, display
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import time
from google.colab import files


#Define Utility Functions

In [91]:
# 💬 Utility Functions

def show_loading_animation():
    """Display a simple loading animation in the output area"""
    with output_area:
        for _ in range(3):
            for dots in [".", "..", "..."]:
                clear_output(wait=True)
                display(HTML(f"<div style='color:#666;'><i>Processing{dots}</i></div>"))
                time.sleep(0.3)

def detect_question_type(question):
    question_lower = question.lower()
    if any(term in question_lower for term in [
        "analyze", "analysis", "performance", "how did", "how well",
        "financial results", "compare", "trend", "growth"
    ]):
        return "financial_analysis"
    if any(term in question_lower for term in [
        "invest", "stock", "share", "dividend", "return", "portfolio",
        "should i", "worth", "recommendation", "outlook"
    ]):
        return "investment"
    if any(pattern in question_lower for pattern in [
        "what is", "what are", "define", "explain", "meaning of", "definition",
        "describe", "elaborate on", "tell me about", "how does", "concept of"
    ]):
        return "term_explanation"
    return "general"


#Define Core Assistant Logic

In [92]:
# 🧠 Core Assistant Logic

def enhanced_query_assistant(question):
    question_type = detect_question_type(question)
    result = qa_chain({"query": question})

    sources = []
    sources_seen = set()
    for doc in result["source_documents"]:
        source = doc.metadata.get("source", "Unknown")
        page = doc.metadata.get("page", "")
        page_info = f" (Page {page})" if page != "" else ""
        source_citation = f"{source}{page_info}"
        if source_citation not in sources_seen:
            sources_seen.add(source_citation)
            sources.append(source_citation)

    return {
        "answer": result["result"],
        "sources": sources[:3],
        "question_type": question_type
    }


#Define UI Widgets

In [93]:
# 🎨 Widgets Setup

question_input = widgets.Text(
    value='',
    placeholder='Ask a financial question...',
    description='Question:',
    layout=widgets.Layout(width='70%')
)

ask_button = widgets.Button(
    description='Ask',
    button_style='primary',
    tooltip='Ask your question',
    layout=widgets.Layout(width='15%')
)

clear_button = widgets.Button(
    description='Clear Chat',
    button_style='danger',
    tooltip='Clear chat history',
    layout=widgets.Layout(width='15%')
)

output_area = widgets.Output(
    layout={'border': '1px solid #ddd', 'padding': '10px', 'height': '400px', 'overflow': 'auto'}
)

chat_summary_button = widgets.Button(
    description='View Chat Summary',
    button_style='info',
    tooltip='View all chat history'
)


#Button Functions

In [94]:
# 🛠 Button Handling Functions

if 'chat_history' not in globals():
    global chat_history
    chat_history = []

def on_ask_clicked(b):
    question = question_input.value
    if not question.strip():
        return
    question_input.value = ''
    with output_area:
        display(HTML(f"<div style='background-color:#f0f0f0; padding:10px;'><b>You:</b> {question}</div>"))
        show_loading_animation()
        try:
            result = enhanced_query_assistant(question)
            clear_output()
            display(HTML(f"<div style='background-color:#f0f0f0; padding:10px;'><b>You:</b> {question}</div>"))

            badge = ""
            if result["question_type"] == "financial_analysis":
                badge = "<span style='background-color:#007bff; color:white; padding:3px 6px;'>Financial Analysis</span>"
            elif result["question_type"] == "investment":
                badge = "<span style='background-color:#28a745; color:white; padding:3px 6px;'>Investment</span>"
            elif result["question_type"] == "term_explanation":
                badge = "<span style='background-color:#ffc107; color:black; padding:3px 6px;'>Term Explanation</span>"
            else:
                badge = "<span style='background-color:#6c757d; color:white; padding:3px 6px;'>General</span>"

            display(HTML(f"<div style='background-color:#e6f7ff; padding:10px;'><b>Financial Assistant{badge}:</b> {result['answer']}</div>"))

            sources_html = ", ".join(result["sources"])
            display(HTML(f"<div style='font-size:12px; color:#666;'>Sources: {sources_html}</div>"))

            chat_history.append({
                "question": question,
                "answer": result["answer"],
                "sources": result["sources"],
                "question_type": result["question_type"]
            })
        except Exception as e:
            clear_output()
            display(HTML(f"<div style='color:red;'>Error: {str(e)}</div>"))

def on_clear_clicked(b):
    with output_area:
        clear_output()
        chat_history.clear()
        display(HTML("<div style='color:#666;'>Chat history cleared.</div>"))

ask_button.on_click(on_ask_clicked)
clear_button.on_click(on_clear_clicked)
chat_summary_button.on_click(lambda b: display_chat_summary())


#Visualization, Summary, Export, Help

In [95]:
# 📊 Visualization, Summary, Help, Export Functions

# Function to display all chat history at once
def display_chat_summary():
    if not chat_history:
        print("No chat history to display.")
        return

    # Create a table to summarize all interactions
    summary_data = []
    for i, exchange in enumerate(chat_history):
        summary_data.append({
            "Q#": i+1,
            "Question": exchange["question"][:50] + "..." if len(exchange["question"]) > 50 else exchange["question"],
            "Type": exchange["question_type"].replace("_", " ").title(),
            "Sources": ", ".join([s.split(" (Page")[0] for s in exchange["sources"]])
        })

    # Display as DataFrame
    display(HTML("<h3>Chat History Summary</h3>"))
    summary_df = pd.DataFrame(summary_data)
    display(summary_df)

    # Now display the full exchange details
    display(HTML("<h3>Detailed Chat History</h3>"))
    for i, exchange in enumerate(chat_history):
        display(HTML(f"""
        <div style="border:1px solid #ddd; padding:10px; margin-bottom:20px; border-radius:5px;">
            <h4>Q{i+1}: {exchange["question"]}</h4>
            <div style="margin-left:20px;">
                <p><strong>Answer:</strong> {exchange["answer"]}</p>
                <p><strong>Sources:</strong> {", ".join(exchange["sources"])}</p>
                <p><strong>Question Type:</strong> {exchange["question_type"].replace("_", " ").title()}</p>
            </div>
        </div>
        """))

# Create a button to show chat history
chat_summary_button = widgets.Button(
    description='View Chat Summary',
    button_style='info',
    tooltip='View all chat history'
)

def add_visualization_tab():
    """Add a tab for financial data visualization"""
    import matplotlib.pyplot as plt
    import numpy as np

    visualization_output = widgets.Output()

    with visualization_output:
        labels = ['Financial Analysis', 'Investment', 'Term Explanation', 'General']
        counts = [
            sum(1 for item in chat_history if item['question_type'] == 'financial_analysis'),
            sum(1 for item in chat_history if item['question_type'] == 'investment'),
            sum(1 for item in chat_history if item['question_type'] == 'term_explanation'),
            sum(1 for item in chat_history if item['question_type'] == 'general')
        ]

        plt.figure(figsize=(10, 6))
        plt.bar(labels, counts, color=['#007bff', '#28a745', '#ffc107', '#6c757d'])
        plt.title('Questions by Type')
        plt.xlabel('Question Type')
        plt.ylabel('Count')
        plt.tight_layout()
        plt.show()

        display(HTML("<p>This visualization shows the distribution of question types in your conversation history.</p>"))

    return visualization_output

def add_export_functionality():
    """Add functionality to export conversation history"""
    def export_to_markdown():
        """Export chat history to markdown file"""
        markdown_content = "# Financial Content Assistant - Chat History\n\n"
        markdown_content += f"Generated on: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"

        for i, exchange in enumerate(chat_history):
            markdown_content += f"## Q{i+1}: {exchange['question']}\n\n"
            markdown_content += f"**Question Type**: {exchange['question_type'].replace('_', ' ').title()}\n\n"
            markdown_content += f"### Answer:\n\n{exchange['answer']}\n\n"
            markdown_content += f"**Sources**: {', '.join(exchange['sources'])}\n\n"
            markdown_content += "---\n\n"

        # Save to file
        filename = "financial_assistant_history.md"
        with open(filename, "w") as f:
            f.write(markdown_content)

        return filename

    export_button = widgets.Button(
        description='Export to Markdown',
        button_style='success',
        tooltip='Export chat history to markdown'
    )

#Add Help Section
def add_help_section():
    """Add a comprehensive help section"""
    help_output = widgets.Output()

    with help_output:
        display(HTML("""
        <div style="padding: 20px; background-color: #f9f9f9; border-radius: 10px;">
            <h2>Financial Content Assistant - Help</h2>

            <h3>Overview</h3>
            <p>This application uses Retrieval-Augmented Generation (RAG) to provide accurate answers to financial questions based on your uploaded documents.</p>

            <h3>How It Works</h3>
            <ol>
                <li><strong>Document Upload</strong>: Upload financial documents (PDF, TXT) using the file uploader</li>
                <li><strong>Document Processing</strong>: Documents are processed, split into chunks, and stored in a vector database</li>
                <li><strong>Question Analysis</strong>: When you ask a question, the system analyzes the type of question:
                    <ul>
                        <li><span style="background-color:#007bff; color:white; padding:3px 6px; border-radius:3px; font-size:12px;">Financial Analysis</span>: Questions about performance, metrics, or financial results</li>
                        <li><span style="background-color:#28a745; color:white; padding:3px 6px; border-radius:3px; font-size:12px;">Investment</span>: Questions about investments, stocks, or outlook</li>
                        <li><span style="background-color:#ffc107; color:black; padding:3px 6px; border-radius:3px; font-size:12px;">Term Explanation</span>: Questions seeking definitions or explanations</li>
                        <li><span style="background-color:#6c757d; color:white; padding:3px 6px; border-radius:3px; font-size:12px;">General</span>: Other financial questions</li>
                    </ul>
                </li>
                <li><strong>Retrieval</strong>: The system retrieves relevant information from your documents</li>
                <li><strong>Response Generation</strong>: An AI model creates a response based on the retrieved information</li>
                <li><strong>Source Attribution</strong>: The response includes sources from your documents</li>
            </ol>

            <h3>Tips for Best Results</h3>
            <ul>
                <li>Upload comprehensive financial documents like annual reports, financial statements, or analysis reports</li>
                <li>Ask specific questions that relate to the content of your documents</li>
                <li>Use the different tabs to explore your chat history and analyze response patterns</li>
                <li>Review the sources cited to understand where information is coming from</li>
            </ul>
        </div>
        """))

    return help_output


In [96]:
# Function to handle summary button click
def on_summary_clicked(b):
    display_chat_summary()

chat_summary_button.on_click(on_summary_clicked)

# Display the enhanced interface
display(HTML("""
<div style='background-color:#4CAF50; color:white; padding:15px; border-radius:10px; margin-bottom:15px;'>
    <h2 style='margin:0;'>Enhanced Financial Content Assistant</h2>
    <p style='margin:5px 0 0 0;'>Ask questions about your financial documents with specialized response formatting</p>
</div>
"""))

# Display input area
display(widgets.HBox([question_input, ask_button, clear_button]))

# Display output area with welcome message
with output_area:
    display(HTML("<div style='color:#666;'>Welcome to your Enhanced Financial Content Assistant! Ask any question about your uploaded documents.</div>"))

# Create a tabbed interface including visualization and include help
tabs = widgets.Tab()
tab_contents = [output_area, add_visualization_tab(), add_help_section()]
tabs.children = tab_contents
for i, title in enumerate(["Chat", "Visualization", "Help"]):
    tabs.set_title(i, title)
display(tabs)
display(widgets.HBox([chat_summary_button, export_button], layout=widgets.Layout(margin='10px 0 0 0')))


,Q#,Question,Type,Sources
0,1,what is the best profit year?,Term Explanation,"Mircosoft 10-K (1).pdf, Mircosoft 10-K (1).pdf..."
1,2,what is are the potential risks for the netflix,Term Explanation,"Netflix 10-k (2).pdf, Netflix 10-k (2).pdf, Ne..."
2,3,what was the best revenue period?,General,"Netflix 10-k (2).pdf, Netflix 10-k (2).pdf, Ne..."


Exported chat history to financial_assistant_history.md


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Final Launch - Run Interface


In [69]:
# # Create an improved interactive interface for Day 2 - Fixed Version

# # Import necessary libraries
# from IPython.display import clear_output, HTML, display
# import ipywidgets as widgets
# import pandas as pd
# import time
# import datetime

# def show_loading_animation():
#     """Display a simple loading animation in the output area"""
#     import time
#     with output_area:
#         for _ in range(3):
#             for dots in [".", "..", "..."]:
#                 clear_output(wait=True)
#                 display(HTML(f"<div style='color:#666;'><i>Processing{dots}</i></div>"))
#                 time.sleep(0.3)


# # Create a function to detect question type
# def detect_question_type(question):
#     question_lower = question.lower()

#     # Analysis pattern
#     if any(term in question_lower for term in [
#         "analyze", "analysis", "performance", "how did", "how well",
#         "financial results", "compare", "trend", "growth"
#     ]):
#         return "financial_analysis"

#     # Investment pattern
#     if any(term in question_lower for term in [
#         "invest", "stock", "share", "dividend", "return", "portfolio",
#         "should i", "worth", "recommendation", "outlook"
#     ]):
#         return "investment"

#     # Term explanation pattern
#     if any(pattern in question_lower for pattern in [
#         "what is", "what are", "define", "explain", "meaning of", "definition",
#         "describe", "elaborate on", "tell me about", "how does", "concept of"
#     ]):
#         return "term_explanation"

#     # Default
#     return "general"

# # Create an enhanced query function
# def enhanced_query_assistant(question):
#     question_type = detect_question_type(question)

#     # Get the base answer from your existing qa_chain
#     result = qa_chain({"query": question})

#     # Get sources
#     sources = []
#     sources_seen = set()
#     for doc in result["source_documents"]:
#         source = doc.metadata.get("source", "Unknown")
#         page = doc.metadata.get("page", "")
#         page_info = f" (Page {page})" if page != "" else ""

#         source_citation = f"{source}{page_info}"
#         if source_citation not in sources_seen:
#             sources_seen.add(source_citation)
#             sources.append(source_citation)

#     return {
#         "answer": result["result"],
#         "sources": sources[:3],  # Top 3 sources
#         "question_type": question_type
#     }

# # Initialize chat history if it doesn't exist
# if 'chat_history' not in globals():
#     global chat_history
#     chat_history = []

# # Create the UI elements
# question_input = widgets.Text(
#     value='',
#     placeholder='Ask a financial question...',
#     description='Question:',
#     layout=widgets.Layout(width='70%')
# )

# ask_button = widgets.Button(
#     description='Ask',
#     button_style='primary',
#     tooltip='Ask your question',
#     layout=widgets.Layout(width='15%')
# )

# clear_button = widgets.Button(
#     description='Clear Chat',
#     button_style='danger',
#     tooltip='Clear chat history',
#     layout=widgets.Layout(width='15%')
# )

# output_area = widgets.Output(
#     layout={'border': '1px solid #ddd', 'padding': '10px', 'height': '400px', 'overflow': 'auto'}
# )

# # Function to handle ask button click
# def on_ask_clicked(b):
#     question = question_input.value
#     if not question.strip():
#         return

#     # Clear input
#     question_input.value = ''

#     with output_area:
#         # Show user question
#         display(HTML(f"<div style='background-color:#f0f0f0; padding:10px; border-radius:5px; margin-bottom:10px;'><b>You:</b> {question}</div>"))

#         # Show "thinking" message - simpler approach
#         show_loading_animation()

#         try:
#             # Get enhanced answer
#             result = enhanced_query_assistant(question)

#             # Clear output to remove thinking message
#             clear_output()

#             # Re-display user question
#             display(HTML(f"<div style='background-color:#f0f0f0; padding:10px; border-radius:5px; margin-bottom:10px;'><b>You:</b> {question}</div>"))

#             # Create badge based on question type
#             badge = ""
#             if result["question_type"] == "financial_analysis":
#                 badge = "<span style='background-color:#007bff; color:white; padding:3px 6px; border-radius:3px; font-size:12px; margin-left:5px;'>Financial Analysis</span>"
#             elif result["question_type"] == "investment":
#                 badge = "<span style='background-color:#28a745; color:white; padding:3px 6px; border-radius:3px; font-size:12px; margin-left:5px;'>Investment</span>"
#             elif result["question_type"] == "term_explanation":
#                 badge = "<span style='background-color:#ffc107; color:black; padding:3px 6px; border-radius:3px; font-size:12px; margin-left:5px;'>Term Explanation</span>"
#             else:
#                 badge = "<span style='background-color:#6c757d; color:white; padding:3px 6px; border-radius:3px; font-size:12px; margin-left:5px;'>General</span>"

#             # Show answer with badge
#             display(HTML(f"<div style='background-color:#e6f7ff; padding:10px; border-radius:5px; margin-bottom:10px;'><b>Financial Assistant{badge}:</b> {result['answer']}</div>"))

#             # Show sources
#             sources_html = ", ".join(result["sources"])
#             display(HTML(f"<div style='font-size:12px; color:#666; margin-bottom:20px;'>Sources: {sources_html}</div>"))

#             # Add to chat history
#             chat_history.append({
#                 "question": question,
#                 "answer": result["answer"],
#                 "sources": result["sources"],
#                 "question_type": result["question_type"]
#             })

#         except Exception as e:
#             # Clear output and show error
#             clear_output()
#             # Re-display user question
#             display(HTML(f"<div style='background-color:#f0f0f0; padding:10px; border-radius:5px; margin-bottom:10px;'><b>You:</b> {question}</div>"))
#             display(HTML(f"<div style='color:red;'>Error: {str(e)}</div>"))

# # Function to handle clear button click
# def on_clear_clicked(b):
#     with output_area:
#         clear_output()
#         chat_history.clear()
#         display(HTML("<div style='color:#666;'>Chat history cleared.</div>"))

# # Set up button click handlers
# ask_button.on_click(on_ask_clicked)
# clear_button.on_click(on_clear_clicked)

# # Function to display all chat history at once
# def display_chat_summary():
#     if not chat_history:
#         print("No chat history to display.")
#         return

#     # Create a table to summarize all interactions
#     summary_data = []
#     for i, exchange in enumerate(chat_history):
#         summary_data.append({
#             "Q#": i+1,
#             "Question": exchange["question"][:50] + "..." if len(exchange["question"]) > 50 else exchange["question"],
#             "Type": exchange["question_type"].replace("_", " ").title(),
#             "Sources": ", ".join([s.split(" (Page")[0] for s in exchange["sources"]])
#         })

#     # Display as DataFrame
#     display(HTML("<h3>Chat History Summary</h3>"))
#     summary_df = pd.DataFrame(summary_data)
#     display(summary_df)

#     # Now display the full exchange details
#     display(HTML("<h3>Detailed Chat History</h3>"))
#     for i, exchange in enumerate(chat_history):
#         display(HTML(f"""
#         <div style="border:1px solid #ddd; padding:10px; margin-bottom:20px; border-radius:5px;">
#             <h4>Q{i+1}: {exchange["question"]}</h4>
#             <div style="margin-left:20px;">
#                 <p><strong>Answer:</strong> {exchange["answer"]}</p>
#                 <p><strong>Sources:</strong> {", ".join(exchange["sources"])}</p>
#                 <p><strong>Question Type:</strong> {exchange["question_type"].replace("_", " ").title()}</p>
#             </div>
#         </div>
#         """))

# # Create a button to show chat history
# chat_summary_button = widgets.Button(
#     description='View Chat Summary',
#     button_style='info',
#     tooltip='View all chat history'
# )

# def add_visualization_tab():
#     """Add a tab for financial data visualization"""
#     import matplotlib.pyplot as plt
#     import numpy as np

#     visualization_output = widgets.Output()

#     with visualization_output:
#         labels = ['Financial Analysis', 'Investment', 'Term Explanation', 'General']
#         counts = [
#             sum(1 for item in chat_history if item['question_type'] == 'financial_analysis'),
#             sum(1 for item in chat_history if item['question_type'] == 'investment'),
#             sum(1 for item in chat_history if item['question_type'] == 'term_explanation'),
#             sum(1 for item in chat_history if item['question_type'] == 'general')
#         ]

#         plt.figure(figsize=(10, 6))
#         plt.bar(labels, counts, color=['#007bff', '#28a745', '#ffc107', '#6c757d'])
#         plt.title('Questions by Type')
#         plt.xlabel('Question Type')
#         plt.ylabel('Count')
#         plt.tight_layout()
#         plt.show()

#         display(HTML("<p>This visualization shows the distribution of question types in your conversation history.</p>"))

#     return visualization_output

# def add_export_functionality():
#     """Add functionality to export conversation history"""
#     def export_to_markdown():
#         """Export chat history to markdown file"""
#         markdown_content = "# Financial Content Assistant - Chat History\n\n"
#         markdown_content += f"Generated on: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"

#         for i, exchange in enumerate(chat_history):
#             markdown_content += f"## Q{i+1}: {exchange['question']}\n\n"
#             markdown_content += f"**Question Type**: {exchange['question_type'].replace('_', ' ').title()}\n\n"
#             markdown_content += f"### Answer:\n\n{exchange['answer']}\n\n"
#             markdown_content += f"**Sources**: {', '.join(exchange['sources'])}\n\n"
#             markdown_content += "---\n\n"

#         # Save to file
#         filename = "financial_assistant_history.md"
#         with open(filename, "w") as f:
#             f.write(markdown_content)

#         return filename

#     export_button = widgets.Button(
#         description='Export to Markdown',
#         button_style='success',
#         tooltip='Export chat history to markdown'
#     )

#     def on_export_clicked(b):
#         filename = export_to_markdown()
#         print(f"Exported chat history to {filename}")
#         files.download(filename)

#     export_button.on_click(on_export_clicked)

#     return export_button
# export_button = add_export_functionality()


# #Add Help Section
# def add_help_section():
#     """Add a comprehensive help section"""
#     help_output = widgets.Output()

#     with help_output:
#         display(HTML("""
#         <div style="padding: 20px; background-color: #f9f9f9; border-radius: 10px;">
#             <h2>Financial Content Assistant - Help</h2>

#             <h3>Overview</h3>
#             <p>This application uses Retrieval-Augmented Generation (RAG) to provide accurate answers to financial questions based on your uploaded documents.</p>

#             <h3>How It Works</h3>
#             <ol>
#                 <li><strong>Document Upload</strong>: Upload financial documents (PDF, TXT) using the file uploader</li>
#                 <li><strong>Document Processing</strong>: Documents are processed, split into chunks, and stored in a vector database</li>
#                 <li><strong>Question Analysis</strong>: When you ask a question, the system analyzes the type of question:
#                     <ul>
#                         <li><span style="background-color:#007bff; color:white; padding:3px 6px; border-radius:3px; font-size:12px;">Financial Analysis</span>: Questions about performance, metrics, or financial results</li>
#                         <li><span style="background-color:#28a745; color:white; padding:3px 6px; border-radius:3px; font-size:12px;">Investment</span>: Questions about investments, stocks, or outlook</li>
#                         <li><span style="background-color:#ffc107; color:black; padding:3px 6px; border-radius:3px; font-size:12px;">Term Explanation</span>: Questions seeking definitions or explanations</li>
#                         <li><span style="background-color:#6c757d; color:white; padding:3px 6px; border-radius:3px; font-size:12px;">General</span>: Other financial questions</li>
#                     </ul>
#                 </li>
#                 <li><strong>Retrieval</strong>: The system retrieves relevant information from your documents</li>
#                 <li><strong>Response Generation</strong>: An AI model creates a response based on the retrieved information</li>
#                 <li><strong>Source Attribution</strong>: The response includes sources from your documents</li>
#             </ol>

#             <h3>Tips for Best Results</h3>
#             <ul>
#                 <li>Upload comprehensive financial documents like annual reports, financial statements, or analysis reports</li>
#                 <li>Ask specific questions that relate to the content of your documents</li>
#                 <li>Use the different tabs to explore your chat history and analyze response patterns</li>
#                 <li>Review the sources cited to understand where information is coming from</li>
#             </ul>
#         </div>
#         """))

#     return help_output


# # Function to handle summary button click
# def on_summary_clicked(b):
#     display_chat_summary()

# chat_summary_button.on_click(on_summary_clicked)

# # Display the enhanced interface
# display(HTML("""
# <div style='background-color:#4CAF50; color:white; padding:15px; border-radius:10px; margin-bottom:15px;'>
#     <h2 style='margin:0;'>Enhanced Financial Content Assistant</h2>
#     <p style='margin:5px 0 0 0;'>Ask questions about your financial documents with specialized response formatting</p>
# </div>
# """))

# # Display input area
# display(widgets.HBox([question_input, ask_button, clear_button]))

# # Display output area with welcome message
# with output_area:
#     display(HTML("<div style='color:#666;'>Welcome to your Enhanced Financial Content Assistant! Ask any question about your uploaded documents.</div>"))

# # Create a tabbed interface including visualization and include help
# tabs = widgets.Tab()
# tab_contents = [output_area, add_visualization_tab(), add_help_section()]
# tabs.children = tab_contents
# for i, title in enumerate(["Chat", "Visualization", "Help"]):
#     tabs.set_title(i, title)
# display(tabs)
# display(widgets.HBox([chat_summary_button, export_button], layout=widgets.Layout(margin='10px 0 0 0')))

# # Project Documentation
# display(HTML("""
# <div style="padding: 20px; background-color: #f8f9fa; border-radius: 10px; margin-top: 30px;">
#     <h1>Financial Content Assistant - RAG Implementation</h1>

#     <h2>Project Overview</h2>
#     <p>This Financial Content Assistant implements a Retrieval-Augmented Generation (RAG) system
#     with advanced prompt engineering to provide accurate, contextual responses to financial queries
#     based on uploaded documents.</p>

#     <h2>Core Components</h2>
#     <ol>
#         <li><strong>RAG System</strong>: Processes financial documents, creates vector embeddings, and retrieves relevant context</li>
#         <li><strong>Prompt Engineering</strong>: Detects question types and provides specialized responses</li>
#         <li><strong>Interactive UI</strong>: Allows document upload, question answering, and result visualization</li>
#     </ol>

#     <h2>Technical Implementation</h2>
#     <ul>
#         <li>Uses LangChain framework for RAG pipeline</li>
#         <li>FAISS vector database for efficient retrieval</li>
#         <li>Document chunking with 1000 token chunks and 200 token overlap</li>
#         <li>Specialized prompts for different financial question types</li>
#         <li>Interactive UI built with IPython widgets</li>
#     </ul>

#     <h2>Future Improvements</h2>
#     <ul>
#         <li>Support for more document types (CSV, XLSX)</li>
#         <li>Advanced metadata extraction</li>
#         <li>Response quality evaluation</li>
#         <li>Data visualization features</li>
#         <li>Export functionality</li>
#     </ul>
# </div>
# """))

# print("\nEnhanced Financial Content Assistant")
# print("--------------------------------------------------------")
# print("This enhanced version includes:")
# print("1. Automatic detection of financial question types")
# print("2. Specialized formatting based on question type")
# print("3. Improved source attribution")
# print("4. Chat history summary")

In [72]:
def run_enhanced_financial_assistant():
    """Run the enhanced financial assistant with all improvements"""
    # Display header
    display(HTML("""
    <div style='background-color:#4CAF50; color:white; padding:15px; border-radius:10px; margin-bottom:15px;'>
        <h2 style='margin:0;'>Enhanced Financial Content Assistant</h2>
        <p style='margin:5px 0 0 0;'>Ask questions about your financial documents with advanced features</p>
    </div>
    """))

    # Create tabs for different functionality
    tabs = widgets.Tab()

    # Create the main chat interface
    chat_tab = widgets.VBox([
        widgets.HBox([question_input, ask_button, clear_button]),
        output_area,
        widgets.HBox([chat_summary_button, add_export_functionality()],
                    layout=widgets.Layout(margin='10px 0 0 0'))
    ])

    # Add all tabs
    tab_contents = [
        chat_tab,
        add_visualization_tab(),
        add_help_section()
    ]

    tabs.children = tab_contents
    for i, title in enumerate(["Chat", "Visualization", "Help"]):
        tabs.set_title(i, title)

    display(tabs)

    # Display initial welcome message
    with output_area:
        display(HTML("<div style='color:#666;'>Welcome to your Enhanced Financial Content Assistant! Ask any question about your uploaded documents.</div>"))

    print("\nEnhanced Financial Content Assistant")
    print("--------------------------------------------------------")
    print("This enhanced version includes:")
    print("1. Automatic detection of financial question types")
    print("2. Specialized formatting based on question type")
    print("3. Improved source attribution")
    print("4. Data visualization capabilities")
    print("5. Help documentation")
    print("6. Export functionality")

In [73]:
# Run the enhanced financial assistant
if 'qa_chain' in globals() and qa_chain is not None:
    run_enhanced_financial_assistant()
else:
    print("Please run the document processing cells first to set up the RAG system.")
    print("Then run this cell again to launch the enhanced interface.")

TraitError: The 'children' trait of a HBox instance contains an Instance of a TypedTuple which expected a Widget, not the NoneType None.